In [18]:
import numpy as np
import zlib
import socket
import _mgard as mgard


In [19]:
import h5py 
import numpy as np

In [20]:

class Slicer:

    # default 引数
    def __init__(self,filename="/scratch/aoyagir/step1_500_test.h5") -> None:
        self.filename = filename
        self.file = h5py.File(filename, 'r')
        self.dataset = self.file['data']
        print(self.dataset.shape)

    # Access specific elements in the concatenated array
    def access_array_element(self,timestep, x, y, z):
        element = self.dataset[timestep, x, y, z]
        return element

    # Access a subset of the concatenated array
    def slice_multiple_step(self, file, timestep_start, timestep_end, x_start, x_end, y_start, y_end, z_start, z_end):
        subset = self.dataset[timestep_start:timestep_end, x_start:x_end, y_start:y_end, z_start:z_end]
        return subset
    
    def slice_single_step(self, timestep,  x_start, x_end, y_start, y_end, z_start, z_end):
        subset = self.dataset[timestep,  x_start:x_end, y_start:y_end, z_start:z_end]
        retsubset = np.squeeze(subset)
        return retsubset
    # slice sigle step by size
    def get_xyz_offset_by_size(self, size):
        # 100MB -> 「100/(sizeof(float))」個のデータ
        sizeFloat = 4 # byte
        return int((size/sizeFloat)**(1/3))

In [21]:

# slicer instance
slicer = Slicer("/scratch/aoyagir/step1_256_test_0902.h5") 


(257, 1024, 1024, 1024)


In [22]:
# create a file to write the results
import csv
from datetime import datetime

# Get the current date and time
current_time = datetime.now()

# Format the current date and time as a string
timestamp = current_time.strftime("%Y%m%d_%H%M%S")

# timestep to conduct the benchmark
timestep = 0

# Create the file name based on the timestamp
csv_file = f'bench_{timestamp}_timestep_{timestep}_lossless_single.txt'

import csv

header = ['OriginalSizeInByte','CompressedSizeInByte','CompRatio','avg_load_time', 'std_dev_load_time','load_throughput'
            ,'avg_comp_time', 'std_dev_comp_time','comp_throughput', 'avg_decomp_time',
            'std_dev_decomp_time', 'decomp_throughput']

with open(csv_file, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(header)



In [23]:
import zlib
import time 
# Define the number of repetitions and initialize a list for results
num_repetitions = 3
results = []

OriginalSize = 1 * 1024 * 1024  # 1MB in bytes

# Wrap your outer loop with tqdm to create a progress bar
while OriginalSize <= 8000 * 1024 * 1024:  # 4000MB in bytes. 10 iterations
    # Your code here
    load_exe_times = []  # List to store execution times for each repetition
    comp_exe_times = []
    decomp_exe_times = []
    comp_data_size = None

    # get the offset size
    offset = slicer.get_xyz_offset_by_size(OriginalSize)
    print(offset)
    
    # Measure the execution time of the loading
    start_time = time.time()
    original = slicer.slice_single_step(timestep, 0, offset, 0, offset, 0, offset)
    end_time = time.time()
    load_time = end_time - start_time
    load_exe_times.append(load_time)

    for _ in range(num_repetitions):
        # Measure the execution time of compressing
        comp_start_time = time.time()
        compressed_array = zlib.compress(original.tobytes())
        comp_end_time = time.time()
        comp_exe_times.append(comp_end_time - comp_start_time)
        CompressedSize = len(compressed_array)

        # Measure the execution time of decompressing
        decomp_start_time = time.time()
        decompressed_data = zlib.decompress(compressed_array)
        decompressed_array = np.frombuffer(decompressed_data, dtype=np.int32).reshape(original.shape)
        decomp_end_time = time.time()
        decomp_exe_times.append(decomp_end_time - decomp_start_time)

    # Calculate average and standard deviation of execution times
    avg_load_time = np.mean(load_exe_times)
    std_dev_load_time = np.std(load_exe_times)

    avg_comp_time = np.mean(comp_exe_times)
    std_dev_comp_time = np.std(comp_exe_times)

    avg_decomp_time = np.mean(decomp_exe_times)
    std_dev_decomp_time = np.std(decomp_exe_times)

    row_data = [OriginalSize, CompressedSize,OriginalSize/CompressedSize,
                avg_load_time, std_dev_load_time,OriginalSize/avg_load_time,
                avg_comp_time, std_dev_comp_time,OriginalSize/avg_comp_time,
                avg_decomp_time, std_dev_decomp_time, OriginalSize/avg_decomp_time, 
                ]
    # Write the data row
    with open(csv_file, "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(row_data)

    # Double the size
    OriginalSize *= 2
    print(row_data)

# Convert the results list to a NumPy array for easier manipulation
results_array = np.array(results)

63
[1048576, 922814, 1.136280984033619, 0.6103804111480713, 0.0, 1717905.720512429, 0.2031258742014567, 0.1783171345586713, 5162198.090825399, 0.24077526728312174, 0.3285156902293886, 4354998.799635866]
80
[2097152, 1890239, 1.1094639355129166, 0.029191255569458008, 0.0, 71841788.20297785, 0.15894412994384766, 0.0007748275407531652, 13194271.47602676, 0.017340977986653645, 5.7373350357657594e-05, 120936201.0386068]
101
[4194304, 3796874, 1.1046729493788838, 0.04001498222351074, 0.0, 104818339.70516281, 0.31445082028706867, 0.0006539608331922133, 13338505.513106734, 0.03483851750691732, 1.9651181236725025e-05, 120392723.34614138]
127
[8388608, 7513522, 1.1164681490251842, 0.0585787296295166, 0.0, 143202286.10374567, 0.635926882425944, 0.000700711956105609, 13191151.73744347, 0.07275811831156413, 0.004691580349060107, 115294460.53124109]
161
[16777216, 15203472, 1.1035121451205356, 0.08572006225585938, 0.0, 195720996.444484, 1.3431658744812012, 0.005542037592503328, 12490799.77294704, 0.